In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

# NOTE: librosa dependencies apparently require specific versions of numpy, try numpy==1.21.4
import librosa
import librosa.display

import IPython.display

In [4]:
eq_df = pd.read_csv('large_data/eq_harmony_combined.csv')
display(eq_df)

,class,eq_0,eq_10,eq_20,eq_30,eq_40,eq_60,eq_80,eq_120,eq_160,...,eq_10000,eq_15000,eq_20000,crestfactor,salience,harmonic_power,percussive_power,harmonic_hits,percussive_hits,fold
0,children_playing,5.929216,5.157348,5.165380,5.563031,5.046141,5.564920,5.776320,6.222735,5.007891,...,0.808725,0.608560,0.552146,9.255638,1,0.000011,8.845374e-05,150.472464,4,5
1,children_playing,0.393928,0.369925,0.286040,0.234562,0.166241,0.137337,0.112779,0.137284,0.256864,...,5.756093,4.326530,3.608999,10.187929,1,0.002384,7.574148e-06,52.204787,2,10
2,street_music,3.980402,5.639204,8.452841,13.437525,19.415338,35.319351,61.852247,62.117064,69.794119,...,4.442734,3.375018,3.069331,7.897027,1,0.000030,6.712090e-05,332.313043,4,7
3,engine_idling,49.217855,118.563553,413.636333,356.165592,298.137279,250.708116,187.251374,154.515099,123.041750,...,7.791927,5.879618,5.338780,2.837097,1,0.013045,2.409356e-07,132.342029,16,10
4,jackhammer,32.283071,64.291137,87.047839,95.002994,83.547205,78.752242,62.357059,56.717538,47.499636,...,6.121235,4.655034,3.885653,7.351516,1,0.000003,7.250180e-07,77.518617,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8727,drilling,226.917157,131.379454,94.685060,74.815301,52.637036,41.411663,29.911901,26.653218,25.754017,...,22.676630,17.008773,15.437346,13.399563,1,0.000049,1.228996e-05,0.084806,3,8
8728,street_music,1.150492,1.121215,1.201931,1.686691,3.390091,4.478639,7.423455,8.174720,13.878955,...,3.904974,2.969251,2.703057,7.174449,1,0.000084,1.561267e-06,57.223188,5,9
8729,jackhammer,8.035297,7.375843,10.444401,14.958574,30.222577,33.422335,57.759712,66.668939,61.394004,...,20.074697,15.299449,13.888320,7.165302,1,0.000058,8.202671e-07,176.976608,15,1
8730,street_music,6.834364,5.770494,6.197091,6.069610,6.333885,6.242182,5.731609,5.280899,4.798209,...,3.483784,2.613115,2.370264,14.280547,2,0.000007,9.653588e-07,211.814493,3,3


After initial attempts performed relatively poorly, I decided to try to add another classifier object. Crest factor is the ratio of the maximum amplitude of a signal to its root mean square. As such, I expect short, loud sounds like gunshots to exhibit large crest factors.

After initial attempts with the crest factor I noticed that a number of the audio samples have significant "room noise" present. In an attempt to clean this up I'm taking the hilbert transform (extracts the instantaneous amplitude of a signal). Smoothing this transform and dividing by the root mean square power should then amplify the parts of the signal that are large in amplitude, while minimizing the areas that are simply a constant amplitude "hum". Low crest factor systems will be largely unaffected as the root mean square will be similar to the root mean square of the signal for such signals. Finally we ensure that the maximum amplitude of the signal is scaled to be equal to the input signal.

In [5]:
# add crest factor to the data frame
Crfac = np.zeros((len(eq_df),1))
for i in range(len(eq_df)):
    y,sr = load_data(metadata.loc[i])
    k = np.fft.fftfreq(len(y), d=dt)
    hil = np.abs(scipy.signal.hilbert(y))
    hilk = np.fft.fft(hil)
    hilk *= np.exp(-k*k / (2*10**2))
    hil2 = np.fft.ifft(hilk)
    hil2 = np.abs(hil2)
    hil2 *= hil.max() / hil2.max()
    filt_y = y * hil2/np.sqrt((y*y).mean())
    filt_y *= y.max() / filt_y.max()
    y_sq = filt_y*filt_y
    Cr = filt_y.max() / np.sqrt(y_sq.mean())
    Crfac[i] = Cr

eq_df['crestfactor'] = Crfac

NameError: name 'load_data' is not defined

In [ ]:
eq_df2 = eq_df.copy()
eq_df2.replace({'air_conditioner':0, 'car_horn':1, 'children_playing':2, 'dog_bark':3, 'drilling':4,
                'engine_idling':5, 'gun_shot':6, 'jackhammer':7, 'siren':8, 'street_music':9},inplace=True)
# eq_df2.replace({'air_conditioner':0, 'car_horn':0, 'children_playing':0, 'dog_bark':0, 'drilling':0,
#                 'engine_idling':0, 'gun_shot':1, 'jackhammer':0, 'siren':0, 'street_music':0},inplace=True)

Documentation of the dataset suggests not shuffling the dataset. This is because there are a number of audio files that are taken as sections from longer audio files and will result in anomalous results if these are shuffled together. Instead the dataset has included a psuedorandom "fold" category to serve as splits for cross validation. 

In [ ]:
dropfold = 1

In [ ]:
eq_df3 = eq_df2.drop(eq_df2[eq_df2['fold']==dropfold].index)

In [ ]:
eq_df3[eq_df3['fold']==dropfold].values

In [ ]:
eq_df3.drop(columns='fold',inplace=True)

In [ ]:
eq_df3.iloc[1,1:].values

In [ ]:
X_train = eq_df3.iloc[:,1:].values
# X_train = [eq_df3.iloc[:,1:-2].values, eq_df3.iloc[:,-1]]
X_train

In [ ]:
y_train = eq_df3.iloc[:,0].values
y_train

In [ ]:
X_val = eq_df2[eq_df2['fold'] == dropfold].copy()
X_val.drop(columns='fold',inplace=True)
X_val = X_val.iloc[:,1:].values
y_val = eq_df2[eq_df2['fold'] == dropfold]
y_val = y_val.iloc[:,0].values

In [ ]:
X_val

In [ ]:
y_val

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(150,150,150,150,150,), max_iter=50000)

In [ ]:
mlp.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Here we consider the True Positive Rate for the gunshot data as an indicator of the goodness of fit. Interestingly this is often higher when the classifier can classify into multiple categories than simply gunshot/not gunshot.

In [ ]:
# confmat = confusion_matrix(y_train, mlp.predict(X_train))
# acc = 100*confmat[6,6] / (np.sum(confmat[:,6])+np.sum(confmat[6,:]) - confmat[6,6])
# print("Training accuracy of", acc)
# confmat = confusion_matrix(y_val, mlp.predict(X_val))
# acc = 100*confmat[6,6] / (np.sum(confmat[:,6])+np.sum(confmat[6,:]) - confmat[6,6])
# print("Validation accuracy of", acc)
# confmat = confusion_matrix(y_train, mlp.predict(X_train))
# acc = 100*confmat[1,1] / (np.sum(confmat[:,1])+np.sum(confmat[1,:]) - confmat[1,1])
# print("Training accuracy of", acc)
# confmat = confusion_matrix(y_val, mlp.predict(X_val))
# acc = 100*confmat[1,1] / (np.sum(confmat[:,1])+np.sum(confmat[1,:]) - confmat[1,1])
# print("Validation accuracy of", acc)
# confmat = confusion_matrix(y_train, mlp.predict(X_train))
# acc = 100*confmat[1,1] / (np.sum(np.sum(confmat[1,:])))
# print("Training accuracy of", acc)
# confmat = confusion_matrix(y_val, mlp.predict(X_val))
# acc = 100*confmat[1,1] / (np.sum(confmat[1,:]))
confmat = confusion_matrix(y_train, mlp.predict(X_train))
acc = 100*confmat[6,6] / (np.sum(np.sum(confmat[6,:])))
print("Training accuracy of", acc)
confmat = confusion_matrix(y_val, mlp.predict(X_val))
acc = 100*confmat[6,6] / (np.sum(confmat[6,:]))
print("Validation accuracy of", acc)

In [ ]:
pd.DataFrame(confusion_matrix(y_val, mlp.predict(X_val)),
            columns=["predicted "+str(i) for i in range(10)],
            index=["actual "+str(i) for i in range(10)])
# pd.DataFrame(confusion_matrix(y_val, mlp.predict(X_val)),
#             columns=["predicted "+str(i) for i in range(2)],
#             index=["actual "+str(i) for i in range(2)])

As suggested by the dataset the most accurate results for a classifier are acheived when averaged over the different possible test/validation splits.

In [ ]:
accuracy_vec = np.zeros(10)
for dropfold in range(1,11):
    eq_df2 = eq_df.copy()
    eq_df2.replace({'air_conditioner':0, 'car_horn':1, 'children_playing':2, 'dog_bark':3, 'drilling':4,
                'engine_idling':5, 'gun_shot':6, 'jackhammer':7, 'siren':8, 'street_music':9},inplace=True)
#     eq_df2.replace({'air_conditioner':0, 'car_horn':0, 'children_playing':0, 'dog_bark':0, 'drilling':0,
#                     'engine_idling':0, 'gun_shot':1, 'jackhammer':0, 'siren':0, 'street_music':0},inplace=True)

    eq_df3 = eq_df2.drop(eq_df2[eq_df2['fold']==dropfold].index)
    eq_df3.drop(columns='fold',inplace=True)
    X_train = eq_df3.iloc[:,1:].values
    y_train = eq_df3.iloc[:,0].values
    
#     X_val = eq_df2[eq_df2['fold'] == dropfold]
#     X_val = X_val.iloc[:,1:].values
#     y_val = eq_df2[eq_df2['fold'] == dropfold]
#     y_val = y_val.iloc[:,0].values
    
    X_val = eq_df2[eq_df2['fold'] == dropfold].copy()
    X_val.drop(columns='fold',inplace=True)
    X_val = X_val.iloc[:,1:].values
    y_val = eq_df2[eq_df2['fold'] == dropfold]
    y_val = y_val.iloc[:,0].values
    
    mlp = MLPClassifier(hidden_layer_sizes=(150,150,150,150,150,), max_iter=50000)
    mlp.fit(X_train, y_train)
    confmat = confusion_matrix(y_val, mlp.predict(X_val))
#     acc = 100*confmat[6,6] / (np.sum(confmat[:,6])+np.sum(confmat[6,:]) - confmat[6,6])
#     accuracy_vec[dropfold] = np.round(100*accuracy_score(y_train, mlp.predict(X_train)),2)\
#     acc = 100*confmat[1,1] / np.sum(confmat[1,:])
    acc= 100*confmat[6,6] / np.sum(confmat[6,:])
    accuracy_vec[dropfold-1] = acc
    
print(accuracy_vec)
accuracy_vec.mean()